In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from fbprophet import Prophet
import pickle
import warnings
warnings.filterwarnings("ignore")

ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.


In [2]:
df = pd.read_csv('../../data/raw/zillow_data.csv')

In [3]:
def one_zipcode(df, index):
    
    """
    This function pulls the data for one zipcode at a time and retuns a DataFrame for using in Prophet.
    """
    
    series = df.iloc[index]
    series_data = series.iloc[7:]
    df_series = pd.DataFrame(series_data.values, index = series_data.index, columns = ['y'])
    df_series.index = pd.to_datetime(df_series.index, yearfirst = True, format = '%Y-%m')
    df_series['ds'] = df_series.index
    df_series.reset_index(drop = True, inplace = True)
#     df_series['y'] = df_series['y'].astype('int64')
    return df_series

In [4]:
def get_zipcode(df, index):
    
    """
    This function extracts the zipcode from the dataframe at the index. Useful in constructing data dictionanry
    later.
    """
    
    zipcode = df['RegionName'].iloc[index]
    return zipcode

In [5]:
def Prophet_analysis(df):
    
    """
    This function instantiates a Prophet model, fits it to the DataFrame, and predicts values which are returned in 
    a forecast Dataframe.
    """
    
    m = Prophet(seasonality_mode='multiplicative', interval_width=0.95)
    m.fit(df)
    future = m.make_future_dataframe(60, freq = 'M')
    forecast = m.predict(future)
    return forecast

In [6]:
def most_recent_value(df):
    '''
    Finds the most recent median home value for use in future percent calculation.
    '''
    i = -1
    while np.isnan(df.iloc[i]['y']):
        i -= 1

    return df.iloc[i]['y']

In [7]:
def combine_dataframes(df, forecast):
    """This function combines the original dataframe with data from one zipcode and the forecasted values from the
    Prophet analysis into a single DataFrame. It also adds new columns for percent change from the last value in
    the original dataframe and the estimated gross profit."""
    forecast.rename(columns={'yhat': 'y'}, inplace = True)
    forecast_subset = forecast[forecast['ds']>'2018-04-01'][['ds','y', 'yhat_lower', 'yhat_upper']]
    df_appended = df.append(forecast_subset, sort = True)
    df_appended['pct_change'] = ((df_appended['y']-most_recent_value(df))/most_recent_value(df))*100
    df_appended['gross_profit'] = (df_appended['y']-most_recent_value(df))
    return df_appended

In [8]:
def make_dict(df, zipcode):
    """
    This function returns a summary dictionary from the combined dataframe for a given zipcode. This dictionary
    can be appended to a list that will be used to make a final dataframe.
    """
    
    pct_change_1year = df.loc[(df['ds']>'2019-03-31') 
                                       & (df['ds']<'2019-05-01')]['pct_change'].values[0]
    gross_profit_1year = df.loc[(df['ds']>'2019-03-31') 
                                       & (df['ds']<'2019-05-01')]['gross_profit'].values[0]
    pct_change_5year = df.loc[(df['ds']>='2023-03-31')]['pct_change'].values[0]
    gross_profit_5year = df.loc[(df['ds']>='2023-03-31')]['gross_profit'].values[0]
    estimate_1year = df.loc[(df['ds']>'2019-03-31') 
                                       & (df['ds']<'2019-05-01')]['y'].values[0]
    upper_1year = df.loc[(df['ds']>'2019-03-31') 
                                       & (df['ds']<'2019-05-01')]['yhat_upper'].values[0]
    lower_1year = df.loc[(df['ds']>'2019-03-31') 
                                       & (df['ds']<'2019-05-01')]['yhat_lower'].values[0]
    estimate_5year = df.loc[(df['ds']>='2023-03-31')]['y'].values[0]
    upper_5year = df.loc[(df['ds']>='2023-03-31')]['yhat_upper'].values[0]
    lower_5year = df.loc[(df['ds']>='2023-03-31')]['yhat_lower'].values[0]
    zipcode_dict = {'Zipcode': zipcode, 
                    'estimate_1year': estimate_1year,
                    'upper_1year': upper_1year,
                    'lower_1year': lower_1year,
                    'pct_change_1year': pct_change_1year,
                    'estimate_5year': estimate_5year,
                    'upper_5year': upper_5year,
                    'lower_5year': lower_5year,
                    'gross_profit_1year': gross_profit_1year, 
                    'pct_change_5year': pct_change_5year, 
                    'gross_profit_5year': gross_profit_5year}
    return zipcode_dict

In [9]:
def prophet_master(df, start, stop):
    """
    Puts together all the previous functions into one function that will return a list of dictionaries with the 
    results of the prophet predictions. Can control with start/stop.
    """
    preds = []
    for i in range(start, stop):
        zipcode = get_zipcode(df, i)
        forecast = Prophet_analysis(one_zipcode(df, i))
        combined = combine_dataframes(one_zipcode(df, i), forecast)
        preds.append(make_dict(combined, zipcode))
    return preds

In [23]:
#Suggestion: each person run approx. 4900 zipcodes:
#Brent: 0-4900
#Karen: 4901-9801
#Wyatt: 9802-14723

#Assign to new variable, then pickle the variable with this code:

In [17]:
prophet_lists_brent = prophet_master(df, 0, 4900)

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

...

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

In [10]:
len(prophet_lists_brent)

NameError: name 'prophet_lists_brent' is not defined

In [18]:
with open('../../data/processed/prophet_lists_brent', 'wb') as p:
    pickle.dump(prophet_lists_brent, p)

In [29]:
prophet_lists_brent[-1]

{'Zipcode': 37774,
 'estimate_1year': 222757.2417853096,
 'upper_1year': 227962.47497155072,
 'lower_1year': 216973.5609933398,
 'pct_change_1year': -2.084728885578199,
 'estimate_5year': 265897.74486202653,
 'upper_5year': 309820.91833028645,
 'lower_5year': 221548.19853566948,
 'gross_profit_1year': -4742.758214690402,
 'pct_change_5year': 16.878129609681995,
 'gross_profit_5year': 38397.744862026535}

In [11]:
prophet_lists_forgotten_1 = prophet_master(df, 4900, 4901)
prophet_lists_forgotten_2 = prophet_master(df, 9801, 9802)

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [12]:
prophet_lists_forgotten_1

[{'Zipcode': 95354,
  'estimate_1year': 275859.58596661355,
  'upper_1year': 300042.4189631022,
  'lower_1year': 243067.70076772405,
  'pct_change_1year': 8.949283557114358,
  'estimate_5year': 373049.4655694972,
  'upper_5year': 623618.1969584431,
  'lower_5year': 109404.11842869066,
  'gross_profit_1year': 22659.58596661355,
  'pct_change_5year': 47.33391215225008,
  'gross_profit_5year': 119849.4655694972}]

In [13]:
prophet_lists_forgotten_2

[{'Zipcode': 21163,
  'estimate_1year': 502140.02023787255,
  'upper_1year': 525803.063492774,
  'lower_1year': 472770.4894657555,
  'pct_change_1year': 2.939733546099334,
  'estimate_5year': 552327.12353312,
  'upper_5year': 754771.8382936817,
  'lower_5year': 298208.0194774921,
  'gross_profit_1year': 14340.020237872552,
  'pct_change_5year': 13.22819260621567,
  'gross_profit_5year': 64527.12353312003}]

In [14]:
prophet_lists_forgotten_1.extend(prophet_lists_forgotten_2)

In [15]:
prophet_lists_forgotten_1

[{'Zipcode': 95354,
  'estimate_1year': 275859.58596661355,
  'upper_1year': 300042.4189631022,
  'lower_1year': 243067.70076772405,
  'pct_change_1year': 8.949283557114358,
  'estimate_5year': 373049.4655694972,
  'upper_5year': 623618.1969584431,
  'lower_5year': 109404.11842869066,
  'gross_profit_1year': 22659.58596661355,
  'pct_change_5year': 47.33391215225008,
  'gross_profit_5year': 119849.4655694972},
 {'Zipcode': 21163,
  'estimate_1year': 502140.02023787255,
  'upper_1year': 525803.063492774,
  'lower_1year': 472770.4894657555,
  'pct_change_1year': 2.939733546099334,
  'estimate_5year': 552327.12353312,
  'upper_5year': 754771.8382936817,
  'lower_5year': 298208.0194774921,
  'gross_profit_1year': 14340.020237872552,
  'pct_change_5year': 13.22819260621567,
  'gross_profit_5year': 64527.12353312003}]

In [16]:
with open('../../data/processed/prophet_lists_forgotten_pickle', 'wb') as p:
    pickle.dump(prophet_lists_forgotten_1, p)

In [18]:
with open('../../data/processed/prophet_lists_forgotten_pickle','rb') as bp:
    forgotten_test = pickle.load(bp)

In [20]:
forgotten_test

[{'Zipcode': 95354,
  'estimate_1year': 275859.58596661355,
  'upper_1year': 300042.4189631022,
  'lower_1year': 243067.70076772405,
  'pct_change_1year': 8.949283557114358,
  'estimate_5year': 373049.4655694972,
  'upper_5year': 623618.1969584431,
  'lower_5year': 109404.11842869066,
  'gross_profit_1year': 22659.58596661355,
  'pct_change_5year': 47.33391215225008,
  'gross_profit_5year': 119849.4655694972},
 {'Zipcode': 21163,
  'estimate_1year': 502140.02023787255,
  'upper_1year': 525803.063492774,
  'lower_1year': 472770.4894657555,
  'pct_change_1year': 2.939733546099334,
  'estimate_5year': 552327.12353312,
  'upper_5year': 754771.8382936817,
  'lower_5year': 298208.0194774921,
  'gross_profit_1year': 14340.020237872552,
  'pct_change_5year': 13.22819260621567,
  'gross_profit_5year': 64527.12353312003}]